# Query the events, mentions and knowledge table

In [21]:
from collections import Counter
from datetime import date, timedelta
from gdelt import gdelt as gdelt_client
import pandas as pd
import re

In [34]:
def document_has_theme(document, themes=[]):
    document_themes = str(document["Themes"]).split(";")
    for theme in themes:
        if theme in document_themes:
            return True
        
    return False

def query_documents(graphs, mentions, themes=[]):
    mentioned_documents = graphs.loc[graphs["DocumentIdentifier"].isin(mentions["MentionIdentifier"])]
    if 0 < len(themes):
        return mentioned_documents[mentioned_documents.apply(document_has_theme, args=[themes], axis=1)]
    else:
        return mentioned_documents
    
def query_mentions(mentions, events):
    return mentions.loc[mentions["GLOBALEVENTID"].isin(events["GLOBALEVENTID"])]

def query_top_most_events(mentions, events, max_count):
    top_most = mentions["GLOBALEVENTID"].value_counts(sort=True)[:max_count]
    top_most_event_ids = top_most.index.tolist()
    return events.loc[events["GLOBALEVENTID"].isin(top_most_event_ids)]

def query_complex(date_of_interest=date.today(), max_count=3, full_day=True, trusted_sources=[], min_confidence=100, themes=[]):
    client = gdelt_client(version=2)
    events = client.Search(date_of_interest.strftime("%Y %m %d"), table="events", coverage=full_day)
    mentions = client.Search(date_of_interest.strftime("%Y %m %d"), table="mentions", coverage=full_day)
    graphs = client.Search(date_of_interest.strftime("%Y %m %d"), table="gkg", coverage=full_day) 
    del client
    
    if 0 < len(trusted_sources):
        trusted_mentions = mentions.loc[(mentions["MentionSourceName"].isin(trusted_sources)) & (mentions["Confidence"] >= min_confidence)]
    else:
        trusted_mentions = mentions.loc[mentions["Confidence"] >= min_confidence]
    
    # Top most events can be empty, if the mentioned events occurred not at the same day
    top_most_events = query_top_most_events(trusted_mentions, events, max_count)
    events_mentions = query_mentions(trusted_mentions, top_most_events)
    events_documents = query_documents(graphs, events_mentions, themes)
    return (top_most_events,  events_mentions, events_documents)

def describe_gdelt_result(gdelt_result):
    (events, mentions, documents) = gdelt_result    
    document_source = documents["SourceCommonName"].values
    document_extras = documents["Extras"].values
    document_image_urls = documents["SharingImage"].values
    document_themes = documents["V2Themes"].values
    news = []
    unique_links = {}
    document_index = 0
    for document_extra in document_extras:
        themes_count = {}
        themes = document_themes[document_index].split(";")
        for theme_entry in themes:
            theme_entry_array = theme_entry.split(",")
            if 2 == len(theme_entry_array):
                theme = theme_entry_array[0]
                if theme in themes_count:
                    themes_count[theme] += 1
                else:
                    themes_count[theme] = 1
        theme_counter = Counter(themes_count)
        most_common_themes = theme_counter.most_common(1)
        
        news_entry = {
            "title": None,
            "theme": most_common_themes[0][0] if 0 < len(most_common_themes) else None,
            "source": document_source[document_index],
            "links": [],
            "image": None
        }
        
        for title_match in re.finditer("<PAGE_TITLE>(.+)</PAGE_TITLE>", document_extra):
            (title,) = title_match.groups(1)
            news_entry["title"] = title
        for links_match in re.finditer("<PAGE_LINKS>(.+)</PAGE_LINKS>", document_extra):
            (links,) = links_match.groups(1)
            links = links.split(";")
            for link in links:
                if re.match("https?://", link):
                    if not link in unique_links:
                        news_entry["links"].append(link)
                        unique_links[link] = link
        
        document_image_url = document_image_urls[document_index]
        if re.match("https?://", str(document_image_url)):
            news_entry["image"] = document_image_url
        
        if (not None is news_entry["title"] 
            and not None is news_entry["source"] 
            and 0 < len(news_entry["links"]) 
            and not None is news_entry["image"]):
            news.append(news_entry)
        document_index += 1
    
    return { 
        "locations": events["ActionGeo_FullName"].values,
        "news": news
    }
    
def gdelt_result_tohtml(gdelt_result):
    description = describe_gdelt_result(gdelt_result)
    locations = description["locations"]
    html_news = ""
    html_news += "<p>Locations: "
    for location in locations:
        html_news += "{} ".format(location)
    html_news += "</p>"
    
    news = description["news"]
    for news_entry in news:
        html_news += "<p><b>{}</b></p>".format(news_entry["title"])
        html_news += "<p>Theme: {}</p>".format(news_entry["theme"])
        html_news += "<p>Source: {}</p>".format(news_entry["source"])
        html_news += "<p><img src='{}'/></p>".format(news_entry["image"])
        html_news += "<p>Links:".format(news_entry["source"])
        html_news += "<ul>"
        for link in news_entry["links"]:
            html_news += "<li><a href='{0}'>{0}</a></li>".format(link)
        html_news += "</ul></p>"
    return html_news

In [35]:
trusted_sources = ["bbc.co.uk",
                   "cbsnews.com",
                   "dailymail.co.uk",
                   "latimes.com",
                   "msn.com",
                   "nytimes.com",
                   "reuters.com",
                   "sputniknews.com",
                   "swissinfo.ch",
                   "usatoday.com",
                   "washingtonpost.com",
                   "washingtontimes.com",
                   "yahoo.com"]
themes = []
yesterday = date.today()-timedelta(days=1)
gdelt_result = query_complex(yesterday, max_count=3, full_day=True, trusted_sources=trusted_sources, themes=themes)
describe_gdelt_result(gdelt_result)

{'locations': array(['Washington, District of Columbia, United States',
        'California, United States', 'Ukraine'], dtype=object),
 'news': [{'image': 'https://static.reuters.com/resources/r/?m=02&d=20201031&t=2&i=1539509277&r=LYNXMPEG9U09B',
   'links': ['http://thomsonreuters.com/en/about-us/trust-principles.html',
    'https://www.reuters.com/journalists/will-dunham'],
   'source': 'reuters.com',
   'theme': 'LEADER',
   'title': 'A longtime fixture in U.S. politics, Biden seeks to win elusive prize'}]}

In [29]:
for v2_themes in gdelt_result[2]["V2Themes"]:
    themes_count = {}
    themes = v2_themes.split(";")
    for theme_entry in themes:
        theme_entry_array = theme_entry.split(",")
        if 2 == len(theme_entry_array):
            theme = theme_entry_array[0]
            if theme in themes_count:
                themes_count[theme] += 1
            else:
                themes_count[theme] = 1
    theme_counter = Counter(themes_count)
    most_common_themes = theme_counter.most_common(1)
    main_theme = most_common_themes[0][0] if 0 < len(most_common_themes) else None
    print(main_theme)

None
None
None
None
None
None
None
None
None


In [55]:
gdelt_result[2]["SharingImage"].values[0:5]

array([nan,
       'https://s2.reutersmedia.net/resources/r/?m=02&d=20200821&t=2&i=1530485786&r=LYNXMPEG7K0TC',
       'https://s1.reutersmedia.net/resources/r/?m=02&d=20200821&t=2&i=1530487058&r=LYNXMPEG7K0U2',
       'https://s4.reutersmedia.net/resources/r/?m=02&d=20200821&t=2&i=1530487329&r=LYNXMPEG7K0U9',
       'https://www.swissinfo.ch/resource/image/45982994/landscape_ratio16x9/1920/1080/23f05750e647b8f1409fbde3dfbcf74a/LO/image_kbn25h19o.jpg'],
      dtype=object)

In [122]:
describe_gdelt_result(gdelt_result)

{'locations': array(['Bamako, Bamako, Mali'], dtype=object),
 'news': [{'image': 'https://s2.reutersmedia.net/resources/r/?m=02&d=20200821&t=2&i=1530485786&r=LYNXMPEG7K0TC',
   'links': ['http://thomsonreuters.com/en/about-us/trust-principles.html'],
   'source': 'reuters.com',
   'title': 'U.N. team meets detained Mali president as coup supporters plan to rally'},
  {'image': 'https://s.yimg.com/uu/api/res/1.2/ApRWaFNJfrafM0vVD.hpMQ--~B/aD01NzY7dz0xMDI0O3NtPTE7YXBwaWQ9eXRhY2h5b24-/https://media.zenfs.com/en/france_24_english_articles_100/01b8407d87873d0818657e38244ee12e',
   'links': ['http://www.france24.com/en/20200818-ecowas-calls-on-mali-soldiers-to-end-the-mutiny',
    'http://www.france24.com/en/20200820-a-transitional-civilian-or-military-president-will-take-power-in-mali-junta-tells-france-24',
    'http://www.france24.com/en/20200821-un-team-meets-mali-s-deposed-president-keita-following-coup',
    'http://www.france24.com/en/tag/Ibrahim-Boubacar-Keita/',
    'http://www.fran

In [33]:
from IPython.display import display, HTML
html_news = gdelt_result_tohtml(gdelt_result)
display(HTML(html_news))

In [126]:
html_news

"<p>Locations: Bamako, Bamako, Mali </p><p><b>U.N. team meets detained Mali president as coup supporters plan to rally</b></p><p>Source: reuters.com</p><p><img src='https://s2.reutersmedia.net/resources/r/?m=02&d=20200821&t=2&i=1530485786&r=LYNXMPEG7K0TC'/></p><p>Links:<ul><li><a href='http://thomsonreuters.com/en/about-us/trust-principles.html'>http://thomsonreuters.com/en/about-us/trust-principles.html</a></li></ul></p><p><b>US halts military ties with Mali as crowds hail junta takeover</b></p><p>Source: yahoo.com</p><p><img src='https://s.yimg.com/uu/api/res/1.2/ApRWaFNJfrafM0vVD.hpMQ--~B/aD01NzY7dz0xMDI0O3NtPTE7YXBwaWQ9eXRhY2h5b24-/https://media.zenfs.com/en/france_24_english_articles_100/01b8407d87873d0818657e38244ee12e'/></p><p>Links:<ul><li><a href='http://www.france24.com/en/20200818-ecowas-calls-on-mali-soldiers-to-end-the-mutiny'>http://www.france24.com/en/20200818-ecowas-calls-on-mali-soldiers-to-end-the-mutiny</a></li><li><a href='http://www.france24.com/en/20200820-a-trans

In [129]:
gdelt_result[2]

,GKGRECORDID,DATE,SourceCollectionIdentifier,SourceCommonName,DocumentIdentifier,Counts,V2Counts,Themes,V2Themes,Locations,...,GCAM,SharingImage,RelatedImages,SocialImageEmbeds,SocialVideoEmbeds,Quotations,AllNames,Amounts,TranslationInfo,Extras
42891,20200821074500-259,20200821074500,1,reuters.com,https://uk.reuters.com/article/us-facebook-ind...,NaN,NaN,TAX_ETHNICITY;TAX_ETHNICITY_INDIAN;USPEC_POLIT...,"IDEOLOGY,2688;HATE_SPEECH,533;HATE_SPEECH,1399...",1#United States#US#US#39.828175#-98.5795#US;1#...,...,"wc:393,c12.1:29,c12.10:33,c12.11:1,c12.12:13,c...",https://s1.reutersmedia.net/resources/r/?m=02&...,NaN,NaN,https://youtube.com/user/ReutersVideo;https://...,1588|59||would damage the company business pro...,"Ankhi Das,476;Wall Street Journal,1412;Congres...","2,before the information technology,536;300000...",NaN,<PAGE_LINKS>http://thomsonreuters.com/en/about...
43966,20200821080000-23,20200821080000,1,reuters.com,https://uk.reuters.com/article/uk-facebook-ind...,NaN,NaN,TAX_ETHNICITY;TAX_ETHNICITY_INDIAN;USPEC_POLIT...,"IDEOLOGY,2694;HATE_SPEECH,539;HATE_SPEECH,1405...",1#United States#US#US#39.828175#-98.5795#US;1#...,...,"wc:393,c12.1:29,c12.10:33,c12.11:1,c12.12:13,c...",https://s2.reutersmedia.net/resources/r/?m=02&...,NaN,NaN,https://youtube.com/user/ReutersVideo;https://...,1588|59||would damage the company business pro...,"Ankhi Das,477;Wall Street Journal,1413;Congres...","2,before the information technology,541;300000...",NaN,<PAGE_LINKS>http://thomsonreuters.com/en/about...
45172,20200821080000-1229,20200821080000,1,reuters.com,https://www.reuters.com/article/us-facebook-in...,NaN,NaN,TAX_ETHNICITY;TAX_ETHNICITY_INDIAN;USPEC_POLIT...,"IDEOLOGY,2688;HATE_SPEECH,533;HATE_SPEECH,1399...",1#United States#US#US#39.828175#-98.5795#US;1#...,...,"wc:393,c12.1:29,c12.10:33,c12.11:1,c12.12:13,c...",https://s1.reutersmedia.net/resources/r/?m=02&...,NaN,NaN,https://youtube.com/user/ReutersVideo;https://...,1588|59||would damage the company business pro...,"Ankhi Das,476;Wall Street Journal,1412;Congres...","2,before the information technology,536;300000...",NaN,<PAGE_LINKS>http://thomsonreuters.com/en/about...
47658,20200821084500-966,20200821084500,1,swissinfo.ch,https://www.swissinfo.ch/eng/india-parliamenta...,NaN,NaN,TAX_ETHNICITY;TAX_ETHNICITY_INDIAN;USPEC_POLIT...,"HATE_SPEECH,471;HATE_SPEECH,1337;EPU_POLICY_PO...",1#United States#US#US#39.828175#-98.5795#US;1#...,...,"wc:376,c12.1:29,c12.10:31,c12.11:1,c12.12:12,c...",https://www.swissinfo.ch/resource/image/459824...,NaN,NaN,https://youtube.com/c/swissinfo;,1516|59||would damage the company business pro...,"Aditya Kalra,78;Ankhi Das,414;Wall Street Jour...","2,before the information technology,482;300000...",NaN,<PAGE_TITLE>India parliamentary panel to quest...
48779,20200821083000-666,20200821083000,1,msn.com,https://www.msn.com/en-us/news/technology/indi...,NaN,NaN,MEDIA_SOCIAL;TAX_ETHNICITY;TAX_ETHNICITY_INDIA...,"HATE_SPEECH,645;HATE_SPEECH,1511;EPU_POLICY_PO...",1#United States#US#US#39.828175#-98.5795#US;1#...,...,"wc:406,c12.1:28,c12.10:32,c12.11:1,c12.12:14,c...",NaN,NaN,NaN,NaN,1682|59||would damage the company business pro...,"Dado Ruvic,125;Aditya Kalra,236;Nigam Prusty,2...","2,before the information technology,617;300000...",NaN,<PAGE_TITLE>India parliamentary panel to quest...
48827,20200821083000-714,20200821083000,1,reuters.com,https://af.reuters.com/article/worldNews/idAFK...,NaN,NaN,TAX_ETHNICITY;TAX_ETHNICITY_INDIAN;USPEC_POLIT...,"HATE_SPEECH,539;HATE_SPEECH,1405;EPU_POLICY_PO...",1#United States#US#US#39.828175#-98.5795#US;1#...,...,"wc:386,c12.1:28,c12.10:31,c12.11:1,c12.12:13,c...",https://s2.reutersmedia.net/resources/r/?m=02&...,NaN,NaN,NaN,1588|59||would damage the company business pro...,"Ankhi Das,477;Wall Street Journal,1413;Congres...","2,before the information technology,541;300000...",NaN,<PAGE_ALTURL_AMP>https://mobile.reuters.com/ar...
59793,20200821103000-1378,20200821103000,1,msn.com,http://www.msn.com/en-nz/news/world/firestorm